<a href="https://colab.research.google.com/github/ykim879/whats.todays.trend/blob/master/revised_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import queue
import threading
def eachNewsHeadlines(ID, q):
	response = requests.get('https://newsapi.org/v2/top-headlines?sources=' + ID + '&apiKey=' + key)
	json = response.json()
	if json['status'] != 'ok':
		raise IOError('API key is probably overused :(')
	topNews = []
	topNews.append(ID)
	for elem in json['articles']:
		topNews.append(elem['title'])
	q.put(topNews)

In [3]:
import requests
import pandas as pd
key = '5e74c7cf9e344e3e8c04210f9b75125b'
response = requests.get('https://newsapi.org/v2/sources?'
			   'language=en&country=us&'
			   'apiKey=' + key)
sources = response.json()['sources']
df = pd.DataFrame();
pd.set_option('display.max_columns', 7)
for source in sources:
  row = pd.Series(source)
  df = df.append(row, ignore_index = True)
df.set_index("name", inplace = True)
#collects headline
threads = []
q = queue.Queue()
for id in df['id']:
  t = threading.Thread(target = eachNewsHeadlines, args = (id, q))
  threads.append(t)
  t.start()
for t in threads:
  t.join()
headlines = []
while not q.empty():
  headlines.extend(q.get())

KeyError: 'sources'

In [ ]:
from sklearn.feature_extraction import text
import matplotlib.pyplot as plt
stop_words = text.ENGLISH_STOP_WORDS.union({'says','news','new'})
vectorizer = text.CountVectorizer(stop_words=stop_words, min_df=10)
X = vectorizer.fit_transform(headlines)
keywordDCT = sorted(vectorizer.vocabulary_.items(), key = lambda x:x[1], reverse = True)
keywordDF = pd.DataFrame(keywordDCT, columns = ['keywords', 'counts'])
print(keywordDF[:10])
plt.bar(keywordDF['keywords'][:10], keywordDF['counts'][:10])
plt.title('Top 10 Trends in News')
plt.ylabel('# of Occurances in Headlines')
plt.xlabel('Trends')
plt.show()

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction import text
import matplotlib.pyplot as plt
stop_words = text.ENGLISH_STOP_WORDS
data = pd.read_csv('BBC News Train.csv')
data = data.groupby('Category', sort=True)['Text'].apply(' '.join).reset_index()
r,c = data.shape
documents = []
len_documents = []
y_train = []
for i in range(r):
    document = str(data.loc[i,'Text'])
    y_train.append(str(data.loc[i,'Category']))
    len_documents.append(len(document))
    documents.extend(document)
documents.extend(keywords) #extend keywords
#Form bag of words model using words used at least 10 times
v = text.CountVectorizer(stop_words=stop_words, min_df = 10)
print(documents)
train = vectorizer.fit_transform(documents).toarray()
x_train = []
for i in len(len_documents) - 1:
    x_train.append(train[:len_documents[i], :])#store in train_x
gnb = GaussianNB()
label = gnb.fit(data[x_train, y_train).predict(train[len_documents[-1]:])
print(label)#X_test: keywords, x_train, y_train: trained data